In [1]:
import pandas as pd
import datetime as dt
from datetime import timedelta
from haversine import haversine, Unit


In [15]:
# Read in files
oct_path = "resources/original_resources/JC-202310-citibike-tripdata.csv"
nov_path = "resources/original_resources/JC-202311-citibike-tripdata.csv"
dec_path = "resources/original_resources/JC-202312-citibike-tripdata.csv"

oct_trips = pd.read_csv(oct_path)
nov_trips = pd.read_csv(nov_path)
dec_trips = pd.read_csv(dec_path)

In [16]:
oct_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,56269E94B2206375,classic_bike,2023-10-31 11:52:42,2023-10-31 12:03:33,McGinley Square,JC055,Marin Light Rail,JC013,40.725340,-74.067622,40.714584,-74.042817,member
1,2B2F6514026F5C68,classic_bike,2023-10-16 08:24:33,2023-10-16 08:35:28,Astor Place,JC077,Grove St PATH,JC115,40.719299,-74.071357,40.719410,-74.043090,member
2,3716C0E2166FE548,classic_bike,2023-10-21 11:46:26,2023-10-21 11:56:30,McGinley Square,JC055,Grove St PATH,JC115,40.725428,-74.067577,40.719410,-74.043090,member
3,E06BF0A228598DDF,classic_bike,2023-10-26 09:42:42,2023-10-26 09:51:28,Astor Place,JC077,Marin Light Rail,JC013,40.719267,-74.071328,40.714584,-74.042817,member
4,1EA3E3FFB17B6B16,classic_bike,2023-10-22 13:20:54,2023-10-22 15:00:09,McGinley Square,JC055,Grove St PATH,JC115,40.725340,-74.067622,40.719410,-74.043090,casual


In [17]:
oct_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97584 entries, 0 to 97583
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ride_id             97584 non-null  object 
 1   rideable_type       97584 non-null  object 
 2   started_at          97584 non-null  object 
 3   ended_at            97584 non-null  object 
 4   start_station_name  97579 non-null  object 
 5   start_station_id    97579 non-null  object 
 6   end_station_name    97207 non-null  object 
 7   end_station_id      97207 non-null  object 
 8   start_lat           97584 non-null  float64
 9   start_lng           97584 non-null  float64
 10  end_lat             97497 non-null  float64
 11  end_lng             97497 non-null  float64
 12  member_casual       97584 non-null  object 
dtypes: float64(4), object(9)
memory usage: 9.7+ MB


In [18]:
# Convert to datetime
oct_trips['started_at'] = pd.to_datetime(oct_trips['started_at'])


In [19]:
# Convert to datetime
oct_trips['ended_at'] = pd.to_datetime(oct_trips['ended_at'])

In [20]:
# Calculate time of trip in minutes
oct_trips['trip_length_min'] = (oct_trips['ended_at'] - oct_trips['started_at']).dt.total_seconds() / 60

In [21]:
oct_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_length_min
0,56269E94B2206375,classic_bike,2023-10-31 11:52:42,2023-10-31 12:03:33,McGinley Square,JC055,Marin Light Rail,JC013,40.725340,-74.067622,40.714584,-74.042817,member,10.850000
1,2B2F6514026F5C68,classic_bike,2023-10-16 08:24:33,2023-10-16 08:35:28,Astor Place,JC077,Grove St PATH,JC115,40.719299,-74.071357,40.719410,-74.043090,member,10.916667
2,3716C0E2166FE548,classic_bike,2023-10-21 11:46:26,2023-10-21 11:56:30,McGinley Square,JC055,Grove St PATH,JC115,40.725428,-74.067577,40.719410,-74.043090,member,10.066667
3,E06BF0A228598DDF,classic_bike,2023-10-26 09:42:42,2023-10-26 09:51:28,Astor Place,JC077,Marin Light Rail,JC013,40.719267,-74.071328,40.714584,-74.042817,member,8.766667
4,1EA3E3FFB17B6B16,classic_bike,2023-10-22 13:20:54,2023-10-22 15:00:09,McGinley Square,JC055,Grove St PATH,JC115,40.725340,-74.067622,40.719410,-74.043090,casual,99.250000


In [22]:
# ChatGPT
# Create start and end coordinate locations
oct_trips['start_coord'] = list(zip(oct_trips['start_lat'], oct_trips['start_lng']))
oct_trips['end_coord'] = list(zip(oct_trips['end_lat'], oct_trips['end_lng']))

In [23]:
oct_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_length_min,start_coord,end_coord
0,56269E94B2206375,classic_bike,2023-10-31 11:52:42,2023-10-31 12:03:33,McGinley Square,JC055,Marin Light Rail,JC013,40.725340,-74.067622,40.714584,-74.042817,member,10.850000,"(40.7253399253558, -74.06762212514877)","(40.71458403535893, -74.04281705617905)"
1,2B2F6514026F5C68,classic_bike,2023-10-16 08:24:33,2023-10-16 08:35:28,Astor Place,JC077,Grove St PATH,JC115,40.719299,-74.071357,40.719410,-74.043090,member,10.916667,"(40.719298959, -74.071357369)","(40.71941, -74.04309)"
2,3716C0E2166FE548,classic_bike,2023-10-21 11:46:26,2023-10-21 11:56:30,McGinley Square,JC055,Grove St PATH,JC115,40.725428,-74.067577,40.719410,-74.043090,member,10.066667,"(40.725428224, -74.067576528)","(40.71941, -74.04309)"
3,E06BF0A228598DDF,classic_bike,2023-10-26 09:42:42,2023-10-26 09:51:28,Astor Place,JC077,Marin Light Rail,JC013,40.719267,-74.071328,40.714584,-74.042817,member,8.766667,"(40.719266772, -74.071328402)","(40.71458403535893, -74.04281705617905)"
4,1EA3E3FFB17B6B16,classic_bike,2023-10-22 13:20:54,2023-10-22 15:00:09,McGinley Square,JC055,Grove St PATH,JC115,40.725340,-74.067622,40.719410,-74.043090,casual,99.250000,"(40.7253399253558, -74.06762212514877)","(40.71941, -74.04309)"


In [24]:
# ChatGPT
# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return haversine(point1, point2, unit=Unit.KILOMETERS)

# Calculate distance and add as a column
oct_trips['distance_KM'] = oct_trips.apply(lambda row: calculate_distance(row['start_coord'], row['end_coord']), axis=1)

In [25]:
oct_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_length_min,start_coord,end_coord,distance_KM
0,56269E94B2206375,classic_bike,2023-10-31 11:52:42,2023-10-31 12:03:33,McGinley Square,JC055,Marin Light Rail,JC013,40.725340,-74.067622,40.714584,-74.042817,member,10.850000,"(40.7253399253558, -74.06762212514877)","(40.71458403535893, -74.04281705617905)",2.408412
1,2B2F6514026F5C68,classic_bike,2023-10-16 08:24:33,2023-10-16 08:35:28,Astor Place,JC077,Grove St PATH,JC115,40.719299,-74.071357,40.719410,-74.043090,member,10.916667,"(40.719298959, -74.071357369)","(40.71941, -74.04309)",2.382302
2,3716C0E2166FE548,classic_bike,2023-10-21 11:46:26,2023-10-21 11:56:30,McGinley Square,JC055,Grove St PATH,JC115,40.725428,-74.067577,40.719410,-74.043090,member,10.066667,"(40.725428224, -74.067576528)","(40.71941, -74.04309)",2.169336
3,E06BF0A228598DDF,classic_bike,2023-10-26 09:42:42,2023-10-26 09:51:28,Astor Place,JC077,Marin Light Rail,JC013,40.719267,-74.071328,40.714584,-74.042817,member,8.766667,"(40.719266772, -74.071328402)","(40.71458403535893, -74.04281705617905)",2.458687
4,1EA3E3FFB17B6B16,classic_bike,2023-10-22 13:20:54,2023-10-22 15:00:09,McGinley Square,JC055,Grove St PATH,JC115,40.725340,-74.067622,40.719410,-74.043090,casual,99.250000,"(40.7253399253558, -74.06762212514877)","(40.71941, -74.04309)",2.169989


In [26]:
# Convert to datetime
nov_trips['started_at'] = pd.to_datetime(nov_trips['started_at'])
nov_trips['ended_at'] = pd.to_datetime(nov_trips['ended_at'])

In [27]:
# Calculate time of trip in minutes
nov_trips['trip_length_min'] = (nov_trips['ended_at'] - nov_trips['started_at']).dt.total_seconds() / 60

In [28]:
# Create start and end coordinate locations
nov_trips['start_coord'] = list(zip(nov_trips['start_lat'], nov_trips['start_lng']))
nov_trips['end_coord'] = list(zip(nov_trips['end_lat'], nov_trips['end_lng']))

In [29]:
# Calculate distance and add as a column
nov_trips['distance_KM'] = nov_trips.apply(lambda row: calculate_distance(row['start_coord'], row['end_coord']), axis=1)

In [30]:
nov_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_length_min,start_coord,end_coord,distance_KM
0,1156A0D0E2EB60CB,classic_bike,2023-11-19 12:08:48,2023-11-19 12:28:53,Jackson Square,JC063,Baldwin at Montgomery,JC020,40.711130,-74.078900,40.723659,-74.064194,casual,20.083333,"(40.71113, -74.0789)","(40.7236589, -74.0641943)",1.864653
1,9A6D99BAED3AE106,classic_bike,2023-11-05 18:21:56,2023-11-05 18:25:36,Madison St & 10 St,HB503,Mama Johnson Field - 4 St & Jackson St,HB404,40.750044,-74.035878,40.743140,-74.040041,casual,3.666667,"(40.750044346, -74.035877585)","(40.74313993965626, -74.04004096984863)",0.844056
2,CC3F75A6310A2625,classic_bike,2023-11-12 11:36:41,2023-11-12 11:42:04,Madison St & 10 St,HB503,Mama Johnson Field - 4 St & Jackson St,HB404,40.750100,-74.035852,40.743140,-74.040041,casual,5.383333,"(40.750099778, -74.035852075)","(40.74313993965626, -74.04004096984863)",0.850556
3,3BF4E21F0D5BCB98,classic_bike,2023-11-08 15:04:23,2023-11-08 15:20:23,Jackson Square,JC063,Hilltop,JC019,40.711130,-74.078900,40.731169,-74.057574,member,16.000000,"(40.71113, -74.0789)","(40.7311689, -74.0575736)",2.862717
4,1B7E5ADCABC46F0E,classic_bike,2023-11-22 09:08:22,2023-11-22 09:14:23,Mama Johnson Field - 4 St & Jackson St,HB404,River St & 1 St,HB609,40.743148,-74.040053,40.737215,-74.028865,member,6.016667,"(40.743148446, -74.040052891)","(40.73721535335381, -74.02886540972803)",1.150493


In [31]:
# Convert to datetime
dec_trips['started_at'] = pd.to_datetime(dec_trips['started_at'])
dec_trips['ended_at'] = pd.to_datetime(dec_trips['ended_at'])

In [32]:
# Calculate time of trip in minutes
dec_trips['trip_length_min'] = (dec_trips['ended_at'] - dec_trips['started_at']).dt.total_seconds() / 60

In [33]:
# Create start and end coordinate locations
dec_trips['start_coord'] = list(zip(dec_trips['start_lat'], dec_trips['start_lng']))
dec_trips['end_coord'] = list(zip(dec_trips['end_lat'], dec_trips['end_lng']))

In [34]:
# Calculate distance and add as a column
dec_trips['distance_KM'] = dec_trips.apply(lambda row: calculate_distance(row['start_coord'], row['end_coord']), axis=1)

In [35]:
dec_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_length_min,start_coord,end_coord,distance_KM
0,DAE3C13A5FAAB089,classic_bike,2023-12-29 16:02:20,2023-12-29 16:10:17,Brunswick St,JC023,Communipaw & Berry Lane,JC084,40.724016,-74.050550,40.714358,-74.066611,member,7.950000,"(40.72401619, -74.050550103)","(40.71435836870427, -74.06661093235016)",1.727821
1,1EC6BE46ACF2DE11,classic_bike,2023-12-06 01:58:32,2023-12-06 02:04:48,Brunswick St,JC023,Oakland Ave,JC022,40.724135,-74.050686,40.737604,-74.052478,casual,6.266667,"(40.724134684, -74.050686002)","(40.7376037, -74.0524783)",1.505283
2,105CA439B9CBF57C,classic_bike,2023-12-06 20:15:11,2023-12-06 20:21:02,Brunswick St,JC023,Oakland Ave,JC022,40.723990,-74.050639,40.737604,-74.052478,member,5.850000,"(40.72399044, -74.050639033)","(40.7376037, -74.0524783)",1.521641
3,DC1C46F6090FA1DB,classic_bike,2023-12-09 13:54:19,2023-12-09 13:56:48,Brunswick St,JC023,Manila & 1st,JC082,40.723983,-74.050732,40.721651,-74.042884,casual,2.483333,"(40.723983169, -74.050732374)","(40.72165072487999, -74.04288411140442)",0.710422
4,04EAEAAD6D32A664,classic_bike,2023-12-02 17:30:34,2023-12-02 17:37:26,Brunswick St,JC023,Riverview Park,JC057,40.724011,-74.050686,40.744319,-74.043991,member,6.866667,"(40.724011302, -74.050686479)","(40.7443187, -74.0439909)",2.327489


In [37]:
# # Export the DataFrames as CSV files
# oct_trips.to_csv("resources/cleaned_resources/oct.csv", index=False)
# nov_trips.to_csv("resources/cleaned_resources/nov.csv", index=False)
# dec_trips.to_csv("resources/cleaned_resources/dec.csv", index=False)